In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

import os
import numpy as np
from os import listdir
from pathlib import Path
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
import h5py
import pydicom as dicom
import matplotlib.pyplot as plt
from dicompylercore import dicomparser, dvh, dvhcalc
from dicom_contour.contour import get_contour_file,get_roi_names, coord2pixels, cfile2pixels, plot2dcontour, slice_order, get_contour_dict, get_data,  create_image_mask_files
from preprocessing.get_plan_from_dicom import Plan, plot_DVH, structure_range, img_cut, resample
from preprocessing.get_plan_from_dicom import plot_3d_img, plan_unit_test
from preprocessing.generate_data import get_plans_unit_test
from util import *
from config import *
from preprocessing.generate_data import get_plans
from dicom_contour.contour import get_ct_name_dict
from dicom_contour.dose import build_dose_volume
import pickle
from PIL import Image, ImageDraw
from copy import deepcopy
import math
from tensorflow import keras
#from model_training_evaluation.training import train_nn_model, run_training
from model_training_evaluation.predicting import predict_evaluation, evaluate, predict_batch, predict_unit_test,  metrics_summary
from model_training_evaluation.training import training_unit_test


In [ ]:
## unit test of the plan class
plan = plan_unit_test()

In [ ]:
# Dose Volume histogram plot check
plan.plot_DVH(standard_name)

In [ ]:
# dose normalization check
plan.normalize('Lungs', 8)
plan.plot_DVH(standard_name)

In [ ]:
## generate plans for all plans
get_plans_unit_test()
    


In [ ]:
## training of the models
if __name__ == '__main__':
    run_training(train_gan_model = False, input_size = (16, 64, 128, 12), parent_path='Data', training_npy_path = 'Data/npy_dataset/training/' , validation_npy_path = 'Data/npy_dataset/validation/')

In [ ]:
model_path = 'Data/Model_AttUnetDense_v3/'
if model_path[-1] != '/':
    model_path = model_path + '/'
model =  load_model(model_path + 'final_model.h5' )
model.load_weights(model_path + 'best_weights.h5')
data_folder = 'Data/npy_dataset/test'
data_dirs = listdir(data_folder)

In [ ]:
folder = data_dirs[0]
print('work on test patient ', folder)
predict_evaluation(model, test_patient_path = data_folder+'/'+folder)

In [ ]:
metrics_all = predict_batch(model_path, data_folder)

In [ ]:
metrics_summary(metrics_all)

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

In [ ]:
from numba import cuda
cuda.select_device(0)
cuda.close()

In [ ]:
print(metrics_all.keys())
metrics_all['patient_4']